# 多项选择任务:  
即给一个文本`Context`,然后给一个列表`Choices`,然后让网络选择一个`Choice`。  
本质上是一项分类任务。
# 基于Transformers的实现
## 数据预处理
处理格式：  
[CLS]1 [Context] [SEP] [Question] [Choice1] [SEP]  
[CLS]2 [Context] [SEP] [Question] [Choice2] [SEP]  
[CLS]3 [Context] [SEP] [Question] [Choice3] [SEP]  
[CLS]4 [Context] [SEP] [Question] [Choice4] [SEP]  

以bert模型进行的多项选择任务：
```python
class BertForMultipleChoice(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, 1)

        # Initialize weights and apply final processing
        self.post_init()

    @auto_docstring
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[tuple[torch.Tensor], MultipleChoiceModelOutput]:
        r"""
        input_ids (`torch.LongTensor` of shape `(batch_size, num_choices, sequence_length)`):
            Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are input IDs?](../glossary#input-ids)
        token_type_ids (`torch.LongTensor` of shape `(batch_size, num_choices, sequence_length)`, *optional*):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in `[0,
            1]`:

            - 0 corresponds to a *sentence A* token,
            - 1 corresponds to a *sentence B* token.

            [What are token type IDs?](../glossary#token-type-ids)
        position_ids (`torch.LongTensor` of shape `(batch_size, num_choices, sequence_length)`, *optional*):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range `[0,
            config.max_position_embeddings - 1]`.

            [What are position IDs?](../glossary#position-ids)
        inputs_embeds (`torch.FloatTensor` of shape `(batch_size, num_choices, sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `input_ids` you can choose to directly pass an embedded representation. This
            is useful if you want more control over how to convert `input_ids` indices into associated vectors than the
            model's internal embedding lookup matrix.
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the multiple choice classification loss. Indices should be in `[0, ...,
            num_choices-1]` where `num_choices` is the size of the second dimension of the input tensors. (See
            `input_ids` above)
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        num_choices = input_ids.shape[1] if input_ids is not None else inputs_embeds.shape[1]

        input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None  #[batch_size * num_choices, sequence_length]
        attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        inputs_embeds = (
            inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
            if inputs_embeds is not None
            else None
        )

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1] # [batch * num_choices, hidden_size]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)  # [batch * num_choices, 1]
        reshaped_logits = logits.view(-1, num_choices)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)

        if not return_dict:
            output = (reshaped_logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return MultipleChoiceModelOutput(
            loss=loss,
            logits=reshaped_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
```

# step1 导入相关包

In [49]:
from transformers import  AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

# step2 加载数据集

In [9]:
datasets = load_dataset("../dataset/clue_c3")
datasets.pop("test")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [10]:
datasets['train'][0]

{'id': 0,
 'context': ['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
 'question': '女的最喜欢哪种电影?',
 'choice': ['恐怖片', '爱情片', '喜剧片', '科幻片'],
 'answer': '喜剧片'}

# step3 数据集预处理

In [12]:
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-macbert-base')
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
a = ['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……']

b = '\n'.join(a)
b

'男：你今天晚上有时间吗?我们一起去看电影吧? 女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'

In [29]:
def process_function(examples):
    # 按照上面画的对Question和choice进行配对
    context = []
    q_c     = []
    labels  = []
    for  idx in range(len(examples['context'])):
        ctx = "\n".join(examples['context'][idx])
        que = examples['question'][idx]
        choices = examples['choice'][idx]
        for choice in choices:
            context.append(ctx)
            q_c.append(que + " " + choice)
        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                context.append(ctx)
                q_c.append(que + " " + "padding")
        labels.append(choices.index(examples['answer'][idx])) # 答案的索引
    tokenized_examples = tokenizer(context, q_c, truncation="only_first", max_length=256, padding="max_length")
    # 模型希望的输入是[batch, num_choices, sequence_length] 这里现在是[batch*num_choices, sequence_length]
    tokenized_examples = {k: [v[i:i+4]for i in range(0, len(v),4)]for k, v in tokenized_examples.items()}
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [32]:
tk_ds = datasets.map(function=process_function, batched=True, remove_columns=datasets['train'].column_names)

In [ ]:
tk_ds['train'].column_names

['input_ids', 'token_type_ids', 'attention_mask', 'labels']

In [ ]:
np.array(tk_ds['train'][0:30]['input_ids']).shape  # 数据形状处理正确

(30, 4, 256)

# step4 创建模型

In [41]:
model = AutoModelForMultipleChoice.from_pretrained('hfl/chinese-macbert-base')
model

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

# step5 创建评估函数

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)   

# step6 配置训练参数

In [48]:
args = TrainingArguments(
    output_dir="./muliple_choice",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=32,
    num_train_epochs=3,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    run_name='runs',
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
)
args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
fp16=True,
fp16_

# step7 创建训练器并训练

In [50]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tk_ds['train'],
    eval_dataset=tk_ds['validation'],
    compute_metrics=compute_metrics,
)

In [51]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.050376,0.529350
2,1.206100,0.979254,0.574423
3,0.954200,1.003142,0.585954


TrainOutput(global_step=141, training_loss=1.0065639847559287, metrics={'train_runtime': 1419.0503, 'train_samples_per_second': 25.092, 'train_steps_per_second': 0.099, 'total_flos': 1.873702246273229e+16, 'train_loss': 1.0065639847559287, 'epoch': 3.0})

# step8 模型预测

In [56]:
import torch
class MultipleChoicePipeline:

    def __init__(self, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device
    
    def preprocess(self, context, question, choices):
        ctxs, qcs = [], []
        for choice in choices:
            ctxs.append(context)
            qcs.append(question + " " + choice)
        return self.tokenizer(ctxs, 
                            qcs, 
                            truncation='only_first', 
                            padding="max_length", 
                            max_length=256,
                            return_tensors="pt")

    def prediction(self, inputs):
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        prediction = torch.argmax(logits, dim=-1).cpu().item()
        return choices[prediction]
        
    
    def __call__(self, context, question, choices):
        inputs = self.preprocess(context, question, choices)
        logits = self.prediction(inputs)
        result = self.postprocess(logits, choices)
        return result

In [57]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [ ]:
context = "小明在北京上班，但是他住在河北，所以他每天要在两地奔波。"
question = "小明在哪上班"
choices   = ["北京", "河北"]

In [69]:
pipe(context, question, choices)

'从北京到河北'